## LangChain Config

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'


In [2]:

os.environ["LANGCHAIN_PROJECT"]= os.getenv('LANGCHAIN_PROJECT')
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

## MongoDB Config

In [13]:
MONGODB_ATLAS_CLUSTER_URI = os.getenv('MONGODB_ATLAS_CLUSTER_URI')
DB_NAME = os.getenv('DB_NAME')
COLLECTION_NAME = os.getenv('COLLECTION_NAME')
ATLAS_VECTOR_SEARCH_INDEX_NAME = os.getenv('ATLAS_VECTOR_SEARCH_INDEX_NAME')

In [14]:
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch

vector_search = MongoDBAtlasVectorSearch.from_connection_string(
        MONGODB_ATLAS_CLUSTER_URI,
        f"{DB_NAME}.{COLLECTION_NAME}",
        OpenAIEmbeddings(),
        index_name= ATLAS_VECTOR_SEARCH_INDEX_NAME
    )

## HyDE

### Scientific Paper Passage

In [3]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(model="gpt-3.5-turbo", temperature=0) | StrOutputParser() 
)

# Run
question = "How to release suffering?"
generate_docs_for_retrieval.invoke({"question":question})

"Suffering is a complex and multifaceted experience that can manifest in physical, emotional, and psychological forms. In order to effectively release suffering, it is important to understand the underlying causes and mechanisms that contribute to its presence. \n\nOne approach to releasing suffering is through mindfulness-based practices, such as meditation and deep breathing exercises. These techniques can help individuals cultivate a sense of presence and awareness, allowing them to observe their suffering without becoming overwhelmed by it. By developing a non-judgmental attitude towards their suffering, individuals can begin to unravel the layers of pain and discomfort that contribute to their overall sense of distress. \n\nAnother important aspect of releasing suffering is through the cultivation of self-compassion and self-care. By treating oneself with kindness and understanding, individuals can begin to address the root causes of their suffering and develop healthier coping me

### MongoDB Retrieval

In [9]:

retrieved_docs = vector_search.similarity_search_with_score(query=question,k=3)
retrieved_docs

[(Document(metadata={'_id': {'$oid': '6690ec88f3a760520e8ef9cc'}, 'source': './pdfs/ask me how do I let go of this type of suffering.pdf', 'page': 0, 'start_index': 1535}, page_content="that show up appr opriately to some matter. And as we think about what we're going to do \nwith the matter either we accept it, do something, change it, our emotions and feelings will \nchange with that. The suffering is when we get stuck in something and usually the stuckness \nis because we're not going to take whatever it is and we'll say it shouldn't be this way and it's \neither my fault because of my character or it's the world's fault because it just wasn't going to \ngive it to me. And that piece of suffering is that stuckness  that we get create.  \nThat's the one that we need to learn to let go to watch the belief system and thought process \nthat is side by side with that and start doing reality check, opening it up, learning the skills of \nbeing with what shows up, being with a n emotion an

### Generation

In [10]:
# RAG RAAG
template = """You are an empathetic psychotherapist, parenting counselor, life-coach and mental health expert. 
Be empathetic while answering if the question includes some bad situation. 
Answer the following question based on the context given below, explain each point in a simple language.
Don't look for the answers from other resources. If you don't find the answer from the context, just simply answer: 
"I don't know" or "I didn't understand this, could you please provide some more information on it".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | ChatOpenAI(model="gpt-3.5-turbo",temperature=0)
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retrieved_docs,"question":question})

'To release suffering, you need to follow these steps:\n\n1. Acceptance: Acknowledge that certain things happen in life that are beyond your control, such as the pandemic, loss of a job, or people dying. Understand that these events are a natural part of life and cannot always be prevented.\n\n2. Change negative beliefs: Recognize when you are adding negative beliefs about yourself or the world to a situation, which can lead to suffering. Challenge these beliefs and try to replace them with more positive and realistic thoughts.\n\n3. Feel your emotions: Allow yourself to feel all the uncomfortable emotions that come with a difficult situation. It is normal to experience grief, sadness, anxiety, or shame in response to loss or hardship.\n\n4. Take action: Decide what you can do to address the situation or make changes if possible. Focus on what is within your control and take steps to improve the situation.\n\n5. Practice acceptance: After taking action, learn to accept the things that 